In [1]:
# existing issues with data, to keep in mind...
# zillow data only exists for about 1/2 of neighborhoods (mostly south StL), and not for all months
# weather data is absent for ~12 hours at a time, often

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import geocoder
import seaborn as sns
import time

In [3]:
# load, combine crime data
crimedf = pd.DataFrame(index=[],columns=[])
for counter, file in enumerate(glob.glob('raw_data/stl_crime_data/*.csv')):
    tmp = pd.read_csv(file, encoding = 'ISO-8859-1', skiprows=None, low_memory=False)
    tmp.columns=tmp.columns.str.replace('-','')
    tmp.columns=tmp.columns.str.replace(' ','')
    tmp.columns=tmp.columns.str.replace('DateOccured','Date')
    tmp.columns=tmp.columns.str.replace('DateOccur','Date')
    tmp.columns=tmp.columns.str.replace('ILEADSAddress','Address')
    tmp.columns=tmp.columns.str.replace('ILEADSStreet','Street')
    crimedf = pd.concat([crimedf,tmp], axis=0, ignore_index=True)
# uncomment if keeping only a random sample
# crimedf = crimedf.sample(n=500, axis=0)
crimedf = crimedf.reset_index().drop(['index'],1)

In [4]:
# clean crime data
dropcolumnscrime = ['CADAddress','CADStreet','AdministrativeAdjustmentIndicator','Beat','CodedMonth','Complaint','DateCrimeCoded','FlagAdministrative','FlagCleanup','FlagCrime','FlagUnfounded','ILeadsAdd','ILeadsApprove','ILeadsAsg','ILeadsType','LocationComment','LocationName','MonthReportedtoMSHP','NewCrimeIndicator','UnfoundedCrimeIndicator']
dropcolumnscrime = pd.Index(dropcolumnscrime).intersection(crimedf.columns).tolist()
crimedf = crimedf.drop(dropcolumnscrime, 1)
crimedf = crimedf[crimedf['Count']==1]
crimedf = crimedf.drop('Count', 1)
crimedf['Date'] = pd.to_datetime(crimedf['Date']) # to pandas datetime format
crimedf = crimedf[crimedf['Date'].dt.year>=2008]
# most of these cases correspond to no location (on some street, but no address or neighborhood)
crimedrop = ((crimedf['Address'].isnull()) | (crimedf['Street'].str.contains('UNKNOWN')) | (crimedf['Neighborhood']==0))
crimedf = crimedf[~crimedrop]
print('Dropped ',sum(crimedrop)/crimedf.shape[0]*100,'%',' of crimes',sep='')
# force certain columns to be strings
strcols = ['Crime','District','Description','Address','Street']
crimedf[strcols] = crimedf[strcols].astype(str)
crimedf.loc[crimedf['Crime'].str.len()==5, 'Crime'] = '0'+crimedf.loc[crimedf['Crime'].str.len()==5, 'Crime']
# new columns
crimedf['Year'] = crimedf['Date'].dt.year
crimedf['Month'] = crimedf['Date'].dt.month
crimedf['Day'] = crimedf['Date'].dt.day
crimedf['DayOfWeek'] = crimedf['Date'].dt.dayofweek
crimedf['Hour'] = crimedf['Date'].dt.hour
crimedf['Minute'] = crimedf['Date'].dt.minute
# classify crimes
crimedf['Homocide'] = 0
crimedf['Violent'] = 0
crimedf['Theft'] = 0
crimedf.loc[crimedf['Crime'].str[0:2] == '01', 'Homocide'] = 1
crimedf.loc[(crimedf['Crime'].str[0:2].isin(['02','03','04'])), 'Violent'] = 1
crimedf.loc[(crimedf['Crime'].str[0:2].isin(['03','05','06','07'])), 'Theft'] = 1
crimedf = crimedf.reset_index().drop(['index'],1)

Dropped 0.589867603139% of crimes


In [5]:
# find lat/lon for crimes, use google lookup tool for empty data
import stateplane
from pyproj import Proj, transform

inProj = Proj(init='esri:102696', preserve_units = True)
outProj = Proj(init='epsg:4326')

dropinds = []
for ind in crimedf.index:
    xc = crimedf['XCoord'].iloc[ind]
    yc = crimedf['YCoord'].iloc[ind]
    if (xc==0):
        addrstring = crimedf['Address'].iloc[ind] + ' ' + crimedf['Street'].iloc[ind] + ' St. Louis, MO'
        print("Looking up w/ geocoder.google", addrstring, ind/crimedf.shape[0], sep=' ')
        g = geocoder.google(addrstring)
        if not g.latlng:
            crimedf.loc[ind,'XCoord'], crimedf.loc[ind,'YCoord'] = 906515.1600962189, 1017387.5860900703
            dropinds.append(ind)
        else:
            sp = stateplane.from_latlon(g.latlng[0],g.latlng[1])
            crimedf.loc[ind, 'XCoord'], crimedf.loc[ind, 'YCoord'] = sp[0]/0.3048, sp[1]/0.3048

crimedf['XCoord'], crimedf['YCoord'] = transform(inProj, outProj, crimedf['XCoord'].tolist(), crimedf['YCoord'].tolist())

print('Dropped ',len(dropinds)/crimedf.shape[0],'% of crimes',sep='')
crimedf = crimedf.drop(dropinds)
crimedf = crimedf.reset_index().drop(['index'],1)

Looking up w/ geocoder.google 1224 NORHT 14TH STREET St. Louis, MO 0.0104020506228
Looking up w/ geocoder.google 6717 DELOR St. Louis, MO 0.010411849088
Looking up w/ geocoder.google 5610 VERNON St. Louis, MO 0.0104196878601
Looking up w/ geocoder.google 5610 VERNON St. Louis, MO 0.0104216475531
Looking up w/ geocoder.google 1525 CHOUTEAU St. Louis, MO 0.0104373250974
Looking up w/ geocoder.google 2214 CHEROKEE St. Louis, MO 0.0104471235626
Looking up w/ geocoder.google 5900 LILLIAN St. Louis, MO 0.0104588817208
Looking up w/ geocoder.google 6900 CLAYTON AVE St. Louis, MO 0.010470639879
Looking up w/ geocoder.google 4719 WILCOX St. Louis, MO 0.0105470679073
Looking up w/ geocoder.google 1181 HODIAMONT St. Louis, MO 0.0106058586983
Looking up w/ geocoder.google 1181 HODIAMONT St. Louis, MO 0.0106078183913
Looking up w/ geocoder.google 3631 ALBERTA St. Louis, MO 0.0106352540938
Looking up w/ geocoder.google 5120 KINGSHIGHWAY St. Louis, MO 0.010648971945
Looking up w/ geocoder.google 1906

Looking up w/ geocoder.google 6177 DELMAR St. Louis, MO 0.0340085129065
Looking up w/ geocoder.google 1414 SPRUCE St. Louis, MO 0.0340575052324
Looking up w/ geocoder.google 410 S EUCLID St. Louis, MO 0.0355801867196
Looking up w/ geocoder.google 70 WB AT MADISON St. Louis, MO 0.0359819237915
Looking up w/ geocoder.google 3015 N. 23RD St. Louis, MO 0.0361484976993
Looking up w/ geocoder.google 4337 PRAIRIE AVE St. Louis, MO 0.0361661349366
Looking up w/ geocoder.google 200 EAST GANO AVE St. Louis, MO 0.0364502904265
Looking up w/ geocoder.google 6005 GOODFELLOW BLVD St. Louis, MO 0.0366364612647
Looking up w/ geocoder.google 0 E. COURTOIS / WATER St. Louis, MO 0.0366462597299
Looking up w/ geocoder.google 8600 NTH BROADWAY St. Louis, MO 0.0367246474512
Looking up w/ geocoder.google 200 MARTIN LUTHER KING St. Louis, MO 0.0373439104499
Looking up w/ geocoder.google 3712 N FLORISSANT St. Louis, MO 0.0386549450894
Looking up w/ geocoder.google 0 I 55 NORTHBOUND St. Louis, MO 0.038711776187

Looking up w/ geocoder.google 3500 N NEWSTEAD AVE St. Louis, MO 0.0647972501587
Looking up w/ geocoder.google 1300 AUBERT AVE St. Louis, MO 0.0648188067821
Looking up w/ geocoder.google 3900 SHREVE AVE St. Louis, MO 0.0648403634055
Looking up w/ geocoder.google 5323 NORTHLAND St. Louis, MO 0.0648423230985
Looking up w/ geocoder.google 1012 13TH St. Louis, MO 0.0648501618706
Looking up w/ geocoder.google 1100 BATES St. Louis, MO 0.0648540812567
Looking up w/ geocoder.google 2710 CHOUTEAU St. Louis, MO 0.0648638797219
Looking up w/ geocoder.google 1223 CLINTON St. Louis, MO 0.0648677991079
Looking up w/ geocoder.google 4539 MCMILLAN St. Louis, MO 0.064869758801
Looking up w/ geocoder.google 2940 THOMAS St. Louis, MO 0.0648795572661
Looking up w/ geocoder.google 4370 NEOSHO St. Louis, MO 0.0649246302059
Looking up w/ geocoder.google 3970 DELOR St. Louis, MO 0.0649324689781
Looking up w/ geocoder.google 2800 IOWA St. Louis, MO 0.0649344286711
Looking up w/ geocoder.google 6216 EMMA St. Lou

Looking up w/ geocoder.google 1100 WASHINGTON AVE St. Louis, MO 0.0741312680782
Looking up w/ geocoder.google 0 N 4TH ST St. Louis, MO 0.0742410108881
Looking up w/ geocoder.google 500 N 13TH ST St. Louis, MO 0.0742880435209
Looking up w/ geocoder.google 0 DR MARTIN LUTHER KING DR / HOD St. Louis, MO 0.0743331164606
Looking up w/ geocoder.google 0 HAMPTON AVE / JAMIESON AVE St. Louis, MO 0.0744506980427
Looking up w/ geocoder.google 4247 LAFAYETTE AVE St. Louis, MO 0.0746349091878
Looking up w/ geocoder.google 1100 E GRAND AVE St. Louis, MO 0.074646667346
Looking up w/ geocoder.google 1600 S JEFFERSON AVE St. Louis, MO 0.0746839015137
Looking up w/ geocoder.google 0 HAMPTON AVE / JAMIESON AVE St. Louis, MO 0.0746917402858
Looking up w/ geocoder.google 4051 FEDERER ST St. Louis, MO 0.0747779667793
Looking up w/ geocoder.google 100 N 14TH ST St. Louis, MO 0.074936701915
Looking up w/ geocoder.google 900 MCKINLEY AVE St. Louis, MO 0.0751189533671
Looking up w/ geocoder.google 1800 MARKET 

Looking up w/ geocoder.google 700 CARR LANE DR St. Louis, MO 0.085468092278
Looking up w/ geocoder.google 0 FAIRGROUNDS PARK St. Louis, MO 0.085526883069
Looking up w/ geocoder.google 3500 DUNNICA AVE St. Louis, MO 0.0855366815342
Looking up w/ geocoder.google 500 N 13TH ST St. Louis, MO 0.085638585572
Looking up w/ geocoder.google 700 CARR LANE DR St. Louis, MO 0.0856836585117
Looking up w/ geocoder.google 4500 ALCOTT AVE St. Louis, MO 0.0857953610147
Looking up w/ geocoder.google 4800 NEWBERRY TER St. Louis, MO 0.0858345548753
Looking up w/ geocoder.google 0 S GRAND BLVD / WALSH ST St. Louis, MO 0.0858365145684
Looking up w/ geocoder.google 1000 SPRUCE ST St. Louis, MO 0.0858600308848
Looking up w/ geocoder.google 1900 OLIVE ST St. Louis, MO 0.0858972650524
Looking up w/ geocoder.google 0 COMMERCIAL / LUCAS AVE St. Louis, MO 0.0862323725612
Looking up w/ geocoder.google 700 WILMINGTON AVE St. Louis, MO 0.0864636163391
Looking up w/ geocoder.google 3500 MAGNOLIA AVE St. Louis, MO 0.08

...

Looking up w/ geocoder.google 4333 GOODFELLOW St. Louis, MO 0.927316945074
Looking up w/ geocoder.google 4019 CALIFORNIA St. Louis, MO 0.927328703232
Looking up w/ geocoder.google 2315 MENARD St. Louis, MO 0.927334582311
Looking up w/ geocoder.google 4403 GRAVOIS AVE St. Louis, MO 0.927350259855
Looking up w/ geocoder.google 6253 N ROSEBURY AVE St. Louis, MO 0.927853900965
Looking up w/ geocoder.google 1511 N KINGSHIGHWAY St. Louis, MO 0.927887215747
Looking up w/ geocoder.google 3404 HAMILTON APT 3B St. Louis, MO 0.927897014212
Looking up w/ geocoder.google 610 LOCUST St. Louis, MO 0.927949925924
Looking up w/ geocoder.google 0 ARCH - SOUTH LEG St. Louis, MO 0.927977361626
Looking up w/ geocoder.google 3949 FEDERER St. Louis, MO 0.928255638037
Looking up w/ geocoder.google 1414 SPRUCE St. Louis, MO 0.928618181248
Looking up w/ geocoder.google 1000 DILLON DR St. Louis, MO 0.929858666938
Looking up w/ geocoder.google 1000 DILLON DR St. Louis, MO 0.929860626631
Looking up w/ geocoder.goo

Looking up w/ geocoder.google 0 GRAND AVE / HOLLY HI St. Louis, MO 0.941465928777
Looking up w/ geocoder.google 900 BROADWAY St. Louis, MO 0.94146788847
Looking up w/ geocoder.google 0 1414 S 14TH ST / SPR St. Louis, MO 0.94151296141
Looking up w/ geocoder.google 1315 NORTH KINGSHIGHWAY St. Louis, MO 0.942308596781
Looking up w/ geocoder.google 2800 MERAMEC ST St. Louis, MO 0.94259863135
Looking up w/ geocoder.google 1 THEATER DRIVE St. Louis, MO 0.94288278684
Looking up w/ geocoder.google 1414 SPRUCE St. Louis, MO 0.942902383771
Looking up w/ geocoder.google 420 SOUTH 8TH ST St. Louis, MO 0.943188498954
Looking up w/ geocoder.google 1414 SPRUCE St. Louis, MO 0.943357032554
Looking up w/ geocoder.google 0 GOVERNMENT DR/WELLS St. Louis, MO 0.943413863652
Looking up w/ geocoder.google 925 MANHATTAN St. Louis, MO 0.943511848304
Looking up w/ geocoder.google 260 N SKINKER St. Louis, MO 0.94353536462
Looking up w/ geocoder.google 3041 CARDINAL St. Louis, MO 0.944321201527
Looking up w/ geoc

Looking up w/ geocoder.google 4800 N. BROADWAY St. Louis, MO 0.958374160272
Looking up w/ geocoder.google 4231 HAMMETT PL St. Louis, MO 0.958507419398
Looking up w/ geocoder.google 311 N CARDINAL St. Louis, MO 0.959636202585
Looking up w/ geocoder.google 731 HODIMONT St. Louis, MO 0.959700872455
Looking up w/ geocoder.google 1315 VANDEVENTER St. Louis, MO 0.960237828347
Looking up w/ geocoder.google 0 OFALLON PARK St. Louis, MO 0.96111185144
Looking up w/ geocoder.google 8960 RIVERVIEW BLVD St. Louis, MO 0.961427362018
Looking up w/ geocoder.google 8724 HALLS FERRY CIR St. Louis, MO 0.961597855312
Looking up w/ geocoder.google 1118 7TH St. Louis, MO 0.961599815005
Looking up w/ geocoder.google   MANCHESTER AVE St. Louis, MO 0.961646847638
Looking up w/ geocoder.google 4812 HIGHLAND St. Louis, MO 0.961823220011
Looking up w/ geocoder.google 1414 SPRUCE St. Louis, MO 0.962344498358
Looking up w/ geocoder.google 2000 GRAND St. Louis, MO 0.962763872667
Looking up w/ geocoder.google 9102 HA

Looking up w/ geocoder.google 5200 N NORWOOD DR St. Louis, MO 0.975958485863
Looking up w/ geocoder.google 2700 ANN AVE St. Louis, MO 0.976136817929
Looking up w/ geocoder.google 2 THEATRE DR St. Louis, MO 0.976373940786
Looking up w/ geocoder.google 0 CASS / TUCKER AVE St. Louis, MO 0.976446449428
Looking up w/ geocoder.google 4700 MARGARETTA AVE St. Louis, MO 0.976532675922
Looking up w/ geocoder.google 300 S 18TH ST St. Louis, MO 0.976707088602
Looking up w/ geocoder.google 0 FERRY ST St. Louis, MO 0.976736483997
Looking up w/ geocoder.google 0 HWY I-55 / LOUGHBOROUGH St. Louis, MO 0.976887380361
Looking up w/ geocoder.google 1201 S. 14TH St. Louis, MO 0.977065712427
Looking up w/ geocoder.google 1209 S. 14TH St. Louis, MO 0.977091188436
Looking up w/ geocoder.google 2900 VICTOR ST St. Louis, MO 0.977106865981
Looking up w/ geocoder.google 200 N 9TH ST St. Louis, MO 0.977179374623
Looking up w/ geocoder.google 8500 GAST PL St. Louis, MO 0.977198971553
Looking up w/ geocoder.google 0

Looking up w/ geocoder.google 1200 LUCAS AVE St. Louis, MO 0.988126219909
Looking up w/ geocoder.google 4500 RIVER DES PERES St. Louis, MO 0.988300632589
Looking up w/ geocoder.google 3300 GRAVOIS AVE St. Louis, MO 0.988363342766
Looking up w/ geocoder.google 800 N BROADWAY St. Louis, MO 0.988641619177
Looking up w/ geocoder.google 0 MARYLAND PLZ St. Louis, MO 0.988731765056
Looking up w/ geocoder.google 2700 S JEFFERSON AVE St. Louis, MO 0.988849346638
Looking up w/ geocoder.google 1600 OLIVE ST St. Louis, MO 0.98886306449
Looking up w/ geocoder.google 1200 RUSSELL AVE St. Louis, MO 0.9890570741
Looking up w/ geocoder.google 3900 CHOUTEAU AVE St. Louis, MO 0.989249124017
Looking up w/ geocoder.google 1100 LUCAS AVE St. Louis, MO 0.989458811172
Looking up w/ geocoder.google 0 N BROADWAY @ CARR St. Louis, MO 0.989619506001
Looking up w/ geocoder.google 800 PINE ST St. Louis, MO 0.989823314076
Looking up w/ geocoder.google 700 N 11TH ST St. Louis, MO 0.989923258421
Looking up w/ geocoder

In [22]:
# load and clean weather data
weatherdf = pd.read_csv('raw_data/stl_weather/STL_weather_hourly.csv', encoding = 'ISO-8859-1', skiprows=None, low_memory=False)
dropcolumnsweather = ['PeakWindDirection','HOURLYAltimeterSetting','PeakWindDirection','DAILYAverageSeaLevelPressure','STATION','STATION_NAME','ELEVATION','LATITUDE','LONGITUDE','HOURLYDRYBULBTEMPC','HOURLYWETBULBTEMPC','HOURLYDewPointTempC','HOURLYPressureTendency','HOURLYPressureChange','HOURLYSeaLevelPressure']
weatherdf = weatherdf.drop(dropcolumnsweather, 1)
monthlycols = weatherdf.filter(regex='Monthly|monthly').columns
weatherdf = weatherdf.drop(monthlycols, 1)
dailycols = weatherdf.filter(regex='DAILY').columns
weatherdf = weatherdf.drop(dailycols, 1)
weatherdf = weatherdf[weatherdf['REPORTTPYE']!='SOD']
weatherdf = weatherdf.drop(['REPORTTPYE'],1)
weatherdf['DATE'] = pd.to_datetime(weatherdf['DATE'])
weatherdf['HOURLYDRYBULBTEMPF']=weatherdf['HOURLYDRYBULBTEMPF'].str.replace('s','')
weatherdf['HOURLYWETBULBTEMPF']=weatherdf['HOURLYWETBULBTEMPF'].str.replace('s','')
weatherdf['HOURLYVISIBILITY']=weatherdf['HOURLYVISIBILITY'].str.replace('V','')
weatherdf['HOURLYDewPointTempF']=weatherdf['HOURLYDewPointTempF'].str.replace('s','')
weatherdf['HOURLYWindSpeed']=weatherdf['HOURLYWindSpeed'].str.replace('s','')
weatherdf['HOURLYStationPressure']=weatherdf['HOURLYStationPressure'].str.replace('s','')
weatherdf['HOURLYPrecip']=weatherdf['HOURLYPrecip'].str.replace('s','')
weatherdf['HOURLYPrecip']=weatherdf['HOURLYPrecip'].str.replace('T','.01')
weatherdf['HOURLYSKYCONDITIONS'] = weatherdf['HOURLYSKYCONDITIONS'].str[0:2]
weatherdf.loc[weatherdf['HOURLYSKYCONDITIONS'].isin(['BK','SC','FE','OV','CL','VV']),'HOURLYSKYCONDITIONS'] = np.nan
# weatherdf[weatherdf['HOURLYSKYCONDITIONS'].isin(['BK','SC','FE','OV','CL','VV'])] = np.nan
weatherdf['HOURLYSKYCONDITIONS'].str.replace('VV','OV')
weatherdf = weatherdf[~weatherdf['DATE'].isnull()]
# fill in nan values with previous readings
numcols = ['HOURLYVISIBILITY','HOURLYPRSENTWEATHERTYPE','HOURLYDRYBULBTEMPF','HOURLYWETBULBTEMPF','HOURLYDewPointTempF','HOURLYRelativeHumidity','HOURLYWindSpeed','HOURLYWindDirection','HOURLYWindGustSpeed','HOURLYStationPressure','HOURLYPrecip']
weatherdf[numcols] = weatherdf[numcols].apply(pd.to_numeric, errors='coerce', axis=1)
weatherdf = weatherdf.fillna(method='ffill')
weatherdf = weatherdf.fillna(method='bfill')
weatherdf = weatherdf.reset_index().drop(['index'],1)

In [7]:
# make weather columns in crime dataframe
def find_nearest(array,value):
    ind = (np.abs(array-value)).argmin()
    return ind

crimedftmp = pd.DataFrame(columns=['HOURLYDRYBULBTEMPF','HOURLYWETBULBTEMPF','HOURLYVISIBILITY','HOURLYDewPointTempF','HOURLYRelativeHumidity','HOURLYWindSpeed','HOURLYWindDirection','HOURLYStationPressure','HOURLYPrecip','HOURLYSKYCONDITIONS'])
start = time.time()
for ind in crimedf.index:
    minind = find_nearest(weatherdf['DATE'], crimedf.loc[ind,'Date'])
    weatherdfminind = weatherdf.loc[minind,:]
    crimedftmp = crimedftmp.append(weatherdfminind[['HOURLYDRYBULBTEMPF','HOURLYWETBULBTEMPF','HOURLYVISIBILITY','HOURLYDewPointTempF','HOURLYRelativeHumidity','HOURLYWindSpeed','HOURLYWindDirection','HOURLYStationPressure','HOURLYPrecip','HOURLYSKYCONDITIONS']] )
    print(ind/crimedf.shape[0], sep=' ', end="\r")

end = time.time()
crimedftmp = crimedftmp.reset_index().drop('index',1)
crimedf = pd.merge(crimedf,crimedftmp, left_index=True, right_index=True)
crimedf.columns = ['Address', 'Crime', 'Date', 'Description', 'District', 'Neighborhood','Street', 'XCoord', 'YCoord', 'Year', 'Month', 'Day', 'DayOfWeek','Hour', 'Minute', 'Homocide', 'Violent', 'Theft', 'dry_temp','wet_temp','visibility','dew_point','humidity','wind_speed','wind_dir','pressure','precip','sky']
print('\n')
print('Took', (end-start)/3600, 'hours',sep=' ')

0.999998015026117

Took 3.638757608599133 hours


In [8]:
# load zillow neighborhood data, create zdf dataframe
zndf = pd.DataFrame(index=[],columns=[])
for counter, file in enumerate(glob.glob('raw_data/zillow/*.csv')):
    tmp = pd.read_csv(file, encoding = 'ISO-8859-1', skiprows=None, low_memory=False)
    tmp.columns=tmp.columns.str.replace('-','')
    tmp.columns=tmp.columns.str.replace(' ','')
    tmp = tmp[tmp['City']=='Saint Louis']
    typeval = file.split('/')[-1].split('.')[0]
    tmp['Type'] = typeval
    zndf = pd.concat([zndf,tmp], axis=0, ignore_index=True)
hooddict = {'Carondelet': 1, 'Patch': 2, 'Holly Hills': 3, 'Boulevard Heights': 4, 'Bevo Mill': 5, 'Princeton Heights': 6, 'South Hampton': 7, 'St. Louis Hills': 8, 'Lindenwood Park': 9, 'Ellendale': 10, 'Clifton Heights': 11, 'The Hill': 12, 'Southwest Garden': 13, 'North Hampton': 14, 'Tower Grove South': 15, 'Dutchtown': 16, 'Mount Pleasant': 17, 'Marine Villa': 18, 'Gravois Park': 19, 'Kosciusko': 20, 'Soulard': 21, 'Benton Park': 22, 'McKinley Heights': 23, 'Fox Park': 24, 'Tower Grove East': 25, 'Compton Heights': 26, 'Shaw': 27, 'Botanical Heights (formerly McRee Town)': 28, 'Tiffany': 29, 'Benton Park West': 30, 'The Gate District': 31, 'Lafayette Square': 32, 'Peabody, Darst, Webbe': 33, 'La Salle': 34, 'Downtown': 35, 'Downtown West': 36, 'Midtown': 37, 'Central West End': 38, 'Forest Park Southeast': 39, 'Kings Oak': 40, 'Cheltenham': 41, 'Clayton-Tamm': 42, 'Franz Park': 43, 'Hi-Pointe': 44, 'Wydown-Skinker': 45, 'Skinker-DeBaliviere': 46, 'DeBaliviere Place': 47, 'West End': 48, 'Visitation Park': 49, 'Wells-Goodfellow': 50, 'Academy': 51, 'Kingsway West': 52, 'Fountain Park': 53, 'Lewis Place': 54, 'Kingsway East': 55, 'The Greater Ville': 56, 'The Ville': 57, 'Vandeventer': 58, 'JeffVanderLou': 59, 'St. Louis Place': 60, 'Carr Square': 61, 'Columbus Square': 62, 'Old North St. Louis': 63, 'Near North Riverfront': 64, 'Hyde Park': 65, 'College Hill': 66, 'Fairground Neighborhood': 67, "O'Fallon": 68, 'Penrose': 69, 'Mark Twain-I-70 Industrial': 70, 'Mark Twain': 71, 'Walnut Park East': 72, 'North Point': 73, 'Baden': 74, 'Riverview': 75, 'Walnut Park West': 76, 'Covenant Blu-Grand Center': 77, 'Hamilton Heights': 78, 'North Riverfront': 79, 'Carondelet Park': 80, 'Tower Grove Park': 81, 'Forest Park': 82, 'Fairgrounds Park': 83, 'Penrose Park': 84, "O'Fallon Park": 85, 'Calvary-Bellefontaine Cemetaries': 86}
zndf = zndf.replace({'RegionName': hooddict})
zndf = zndf.drop(['RegionID','City','State','Metro','CountyName','SizeRank','County'],1)
oldcols = zndf.filter(regex='1996|1997|1998|1999|2000|2001|2002|2003|2004|2005|2006|2007|200801|200802|200803|201705|201706').columns
zndf = zndf.drop(oldcols, 1)
zndf = zndf[zndf['Type'].isin(['Neighborhood_MedianValuePerSqft_AllHomes','Neighborhood_PriceToRentRatio_AllHomes','Neighborhood_Turnover_AllHomes','Neighborhood_Zri_AllHomes','Neighborhood_ZriPerSqft_AllHomes','Neighborhood_Zhvi_AllHomes'])]
# zndf = zndf.fillna(method='ffill', axis=1)
# zndf = zndf.fillna(method='bfill', axis=1)
zndf = zndf.reset_index().drop(['index'],1)

zhvidf = zndf[zndf['Type']=='Neighborhood_Zhvi_AllHomes']
zhvidf = pd.melt(zhvidf, id_vars=['RegionName']) 
zhvidf.columns = ['Neighborhood', 'Date', 'ZHVI']
zhvidf = zhvidf[(zhvidf['Date']!='Type')]
zhvidf['Date'] = pd.to_datetime(zhvidf['Date'], format='%Y%m')

zridf = zndf[zndf['Type']=='Neighborhood_Zri_AllHomes']
zridf = pd.melt(zridf, id_vars=['RegionName']) 
zridf.columns = ['Neighborhood', 'Date', 'ZRI']
zridf = zridf[(zridf['Date']!='Type')]
zridf['Date'] = pd.to_datetime(zridf['Date'], format='%Y%m')

zmvsfdf = zndf[zndf['Type']=='Neighborhood_MedianValuePerSqft_AllHomes']
zmvsfdf = pd.melt(zmvsfdf, id_vars=['RegionName']) 
zmvsfdf.columns = ['Neighborhood', 'Date', 'ZRI']
zmvsfdf = zmvsfdf[(zmvsfdf['Date']!='Type')]
zmvsfdf['Date'] = pd.to_datetime(zmvsfdf['Date'], format='%Y%m')

zrisfdf = zndf[zndf['Type']=='Neighborhood_ZriPerSqft_AllHomes']
zrisfdf = pd.melt(zrisfdf, id_vars=['RegionName']) 
zrisfdf.columns = ['Neighborhood', 'Date', 'ZRI']
zrisfdf = zrisfdf[(zrisfdf['Date']!='Type')]
zrisfdf['Date'] = pd.to_datetime(zrisfdf['Date'], format='%Y%m')

zp2rdf = zndf[zndf['Type']=='Neighborhood_PriceToRentRatio_AllHomes']
zp2rdf = pd.melt(zp2rdf, id_vars=['RegionName']) 
zp2rdf.columns = ['Neighborhood', 'Date', 'ZRI']
zp2rdf = zp2rdf[(zp2rdf['Date']!='Type')]
zp2rdf['Date'] = pd.to_datetime(zp2rdf['Date'], format='%Y%m')

ztodf = zndf[zndf['Type']=='Neighborhood_Turnover_AllHomes']
ztodf = pd.melt(ztodf, id_vars=['RegionName']) 
ztodf.columns = ['Neighborhood', 'Date', 'ZRI']
ztodf = ztodf[(ztodf['Date']!='Type')]
ztodf['Date'] = pd.to_datetime(ztodf['Date'], format='%Y%m')

tmp = pd.merge(zhvidf,zridf, on=['Neighborhood','Date'], how='outer')
tmp = pd.merge(tmp,zmvsfdf, on=['Neighborhood','Date'], how='outer')
tmp = pd.merge(tmp,zrisfdf, on=['Neighborhood','Date'], how='outer')
tmp = pd.merge(tmp,zp2rdf, on=['Neighborhood','Date'], how='outer')
zdf = pd.merge(tmp,ztodf, on=['Neighborhood','Date'], how='outer')
zdf.columns = ['Neighborhood', 'Date', 'ZHVI','ZRI','ZMVSF','ZRISF','ZP2R','ZTO']
zdf['Year'] = zdf['Date'].dt.year
zdf['Month'] = zdf['Date'].dt.month
zdf = zdf.drop('Date',1)

In [29]:
# Load neighborhood population data
hoodpopdf = pd.read_csv('raw_data/StL_neighborhood_populations_2010.csv', encoding = 'ISO-8859-1', skiprows=None)
hoodpopdf['Neighborhood'].replace(hooddict, inplace=True)

In [ ]:
# make zillow columns in crime dataframe
crimedf = pd.merge(crimedf, zdf, on=['Neighborhood','Year','Month'], how='left')

In [30]:
# Save as cleaned "pickle" dataframe files
crimedf.to_pickle('cleaned_data/crimedf.pkl')
hoodpopdf.to_pickle('cleaned_data/hoodpopdf.pkl')
zdf.to_pickle('cleaned_data/zdf.pkl')
weatherdf.to_pickle('cleaned_data/weatherdf.pkl')